In [3]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report


from modules.Untangler import Untangler
random_seed = 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
sample_count = 5
df = pd.read_csv("./data/GoldSet.csv")
df["Diff"] = df["Diff"].apply(lambda x: x.replace("\\ No newline at end of file","").strip())

buggy_df = df[df.Decision == 'Buggy'][["CommitMessage", "Diff", "Decision"]]
rows = np.random.choice(buggy_df.index.values,sample_count)
buggy_df = buggy_df.loc[rows]

notbuggy_df = df[df.Decision == 'NotBuggy'][["CommitMessage", "Diff", "Decision"]]
rows = np.random.choice(notbuggy_df.index.values,sample_count)
notbuggy_df = notbuggy_df.loc[rows]

few_shot_data = [*buggy_df.values.tolist() , *notbuggy_df.values.tolist()]
json.dump(few_shot_data,open("./data/FewShots.json","w"))

In [5]:
df = pd.read_csv("./data/GoldSet.csv")
#df["Diff"] = df["Diff"].apply(lambda x: x.replace("\\ No newline at end of file","").strip())

In [32]:
untangler = Untangler("gemini", True)
untangler.prepare_prompt(df["CommitMessage"].iloc[50], df["Diff"].iloc[50])
untangler.print_prompt()

Prompt for: gemini



Commit Messaage: Fix two bugs in MinMaxPriorityQueue (introduced in []). First is a bug in removeAt(int) that sometimes causes the wrong element to be removed. Second is a bug that sometimes causes certain elements to be iterated over more than once if elements were removed during iteration.

Reported externally at https://github.com/google/guava/issues/2658

-------------
Created by MOE: https://github.com/google/moe
MOE_MIGRATED_REVID=140382230
Git Diff:
@@ -1,16 +1,16 @@
-    int getCorrectLastElement(E actualLastElement) {
+    int swapWithConceptuallyLastElement(E actualLastElement) {
       int parentIndex = getParentIndex(size);
       if (parentIndex != 0) {
         int grandparentIndex = getParentIndex(parentIndex);
         int uncleIndex = getRightChildIndex(grandparentIndex);
         if (uncleIndex != parentIndex && getLeftChildIndex(uncleIndex) >= size) {
           E uncleElement = elementData(uncleIndex);
           if (ordering.compare(uncleEleme

In [33]:
df["Decision"].iloc[50]

'Buggy'

In [34]:
untangler.detect()

'The commit message indicates a fix for a bug in `AbstractFuture#toString` in GWT, which caused an incorrect representation due to an exception being thrown during the `getDone()` call. The diff shows that the code now captures the result of `getDone()` in a variable `value` before appending it to the string builder. This ensures that the `getDone()` method is only called once, and any exception thrown during that call is caught and handled appropriately, preventing the incorrect representation. This aligns with the bug fix described in the commit message. The answer is **Buggy**.\n'

In [ ]:
untangler = Untangler("gemini")
y_true, y_pred = untangler.batch_detect(df)
accuracy_score(y_true, y_pred)
print(classification_report(y_true, y_pred))

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

       Buggy       0.72      0.95      0.82       144
    NotBuggy       0.91      0.56      0.69       122

    accuracy                           0.77       266
   macro avg       0.81      0.75      0.75       266
weighted avg       0.80      0.77      0.76       266



In [ ]:
untangler = Untangler("openai")
untangler.get_batch_result("batch_67ba5b015398819094fe8cc1b5928ae6")

In [155]:
untangler = Untangler("openai")

y_true = []
for index, row in df.iterrows():
    y_true.append(row["Decision"])
batch, y_pred = untangler.get_batch_result("batch_67ba558986f08190bb2c61a32d87a2e3")
print("Accuracy: ", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

100%|██████████| 266/266 [00:00<?, ?it/s]


Accuracy:  0.7819548872180451
              precision    recall  f1-score   support

       Buggy       0.73      0.94      0.82       144
    NotBuggy       0.89      0.60      0.72       122

    accuracy                           0.78       266
   macro avg       0.81      0.77      0.77       266
weighted avg       0.81      0.78      0.77       266

